In [1]:
import warnings
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from scipy.optimize import curve_fit
import pandas as pd
from tqdm import tqdm
from astropy.time import Time
from astropy.table import Table
from astropy.units import UnitsWarning


import sys
sys.path.append('../')
from source_names_dict import source_names_dict, source_names_w_counterparts, source_names_readable

In [2]:
warnings.filterwarnings('ignore', category=UnitsWarning, append=True)

In [3]:
def line(x, m, c):
    return m*x+c

In [4]:
def df_properties(tab, tab_name):
    prop = {}
    prop['name']   = tab_name
    prop['length'] = len(tab)
    prop['N_obs']  = len(np.unique(tab["OBSID"]))
    prop['N_bad']  = len(tab[tab['BAD'] == True])
    prop['N_good'] = len(tab[tab['BAD'] == False])
    prop['N_UL']   = len(tab[tab['UL'] == True])
    return prop

In [5]:
def get_exluded_obs(tab_big, tab_small):
    obs1 = np.unique(tab_big['OBSID'])
    obs2 = np.unique(tab_small['OBSID'])
    obs_excluded = np.setdiff1d(obs1,obs2)
    obs_excluded = list(obs_excluded)
    return obs_excluded

In [6]:
res = {} # Dictionary for storing results

include_bad = True
include_UL = True

uvot_rate     = 'COI_SRC_RATE'
uvot_rate_err = 'COI_SRC_RATE_ERR'
xrt_rate      = 'Rate'
xrt_rate_err  = 'Ratepos'

N_mc = 10000

fits_path = '../lightcurves/joined/NAME_NGC_1313_X-1,U,curve_nosys_join.fits'
#fits_path = '../lightcurves/joined/NAME_NGC_1313_X-1,U,hardrat_join,HARD.fits'

if 'curve_nosys' in fits_path:
    xrt_curve = 'FULL'
    xrt_rate_err = 'Ratepos'
elif 'hardrat' in fits_path:
    xrt_curve = fits_path.split('/')[-1].split(',')[-1][:-5]
    xrt_rate_err = 'Error'

uvot_filter = fits_path.split('/')[-1].split(',')[1]
simbad_name = fits_path.split('/')[-1].split(',')[0]
readable_name = source_names_readable[simbad_name]

tab = Table.read(fits_path)
print(tab)

MET,EXTNAME,TSTART,TSTOP,EXPOSURE,TELAPSE,TIME,SRC_AREA,BKG_AREA,STD_AREA,PLATE_SCALE,RAW_TOT_CNTS,RAW_TOT_CNTS_ERR,RAW_BKG_CNTS,RAW_BKG_CNTS_ERR,RAW_STD_CNTS,RAW_STD_CNTS_ERR,RAW_TOT_RATE,RAW_TOT_RATE_ERR,RAW_BKG_RATE,RAW_BKG_RATE_ERR,RAW_STD_RATE,RAW_STD_RATE_ERR,COI_STD_FACTOR,COI_STD_FACTOR_ERR,COI_BKG_FACTOR,COI_BKG_FACTOR_ERR,COI_TOT_RATE,COI_TOT_RATE_ERR,COI_BKG_RATE,COI_BKG_RATE_ERR,COI_SRC_RATE,COI_SRC_RATE_ERR,AP_FACTOR,AP_FACTOR_ERR,AP_COI_SRC_RATE,AP_COI_SRC_RATE_ERR,LSS_FACTOR,LSS_RATE,LSS_RATE_ERR,SENSCORR_FACTOR,SENSCORR_RATE,SENSCORR_RATE_ERR,CORR_RATE,CORR_RATE_ERR,AB_MAG,AB_MAG_ERR,AB_MAG_ERR_STAT,AB_MAG_ERR_SYS,AB_MAG_BKG,AB_MAG_BKG_ERR_STAT,AB_MAG_BKG_ERR_SYS,AB_MAG_BKG_ERR,AB_MAG_LIM,AB_MAG_LIM_SIG,AB_MAG_COI_LIM,AB_FLUX_AA,AB_FLUX_AA_ERR_STAT,AB_FLUX_AA_ERR_SYS,AB_FLUX_AA_ERR,AB_FLUX_AA_BKG,AB_FLUX_AA_BKG_ERR_STAT,AB_FLUX_AA_BKG_ERR_SYS,AB_FLUX_AA_BKG_ERR,AB_FLUX_AA_LIM,AB_FLUX_AA_COI_LIM,AB_FLUX_HZ,AB_FLUX_HZ_ERR_STAT,AB_FLUX_HZ_ERR_SYS,AB_FLUX_HZ_ERR,AB_FLUX_HZ_BKG,AB_FLUX_HZ_BKG_ERR_STAT,AB_FLUX_HZ_BKG_ERR_SYS,AB_FLUX_HZ_BKG_ERR,AB_FLUX_HZ_LIM,AB_FLUX_HZ_COI_LIM,MAG,MAG_ERR,MAG_ERR_STAT,MAG_ERR_SYS,MAG_BKG,MAG_BKG_ERR_STAT,MAG_BKG_ERR_SYS,MAG_BKG_ERR,MAG_LIM,MAG_LIM_SIG,MAG_COI_LIM,FLUX_AA,FLUX_AA_ERR_STAT,FLUX_AA_ERR_SYS,FLUX_AA_ERR,FLUX_AA_BKG,FLUX_AA_BKG_ERR_STAT,FLUX_AA_BKG_ERR_SYS,FLUX_AA_BKG_ERR,FLUX_AA_LIM,FLUX_AA_COI_LIM,FLUX_HZ,FLUX_HZ_ERR_STAT,FLUX_HZ_ERR_SYS,FLUX_HZ_ERR,FLUX_HZ_BKG,FLUX_HZ_BKG_ERR_STAT,FLUX_HZ_BKG_ERR_SYS,FLUX_HZ_BKG_ERR,FLUX_HZ_LIM,FLUX_HZ_COI_LIM,COI_RATE_LIMIT,CORR_RATE_LIMIT,FILTER,RA,DEC,DETX,DETY,NSIGMA,FRAMTIME,SATURATED,SYS_ERR,SRCEXP,BKGEXP,OBSID,MJD_1,YEAR,MJD_0,FLAG_AB_MAG_99,FLAG_UPPER_LIM,FLAG_AB_MAG_ERR>2,FLAG_AB_MAG_>10_<30,MJD_2,T_+ve,T_-ve,Rate,Ratepos,Rateneg,MODE,BAD,UL,BAND
s,,s,s,s,s,s,arcsec2,arcsec2,arcsec2,arcsec / pix,ct,ct,ct,ct,ct,ct,ct / s,ct / s,ct / (arcsec2 s),ct / (arcsec2 s),ct / s,ct / s,,,,,ct / s,ct / s,ct / (arcsec2 s),ct / (arcsec2 s),ct / s,ct / s,,,ct / s,ct / s,,ct / s,ct / s,,ct / s,ct / s,ct / s,ct / s,mag,mag,mag,mag,mag / arcsec2,mag / arcsec2,mag / arcsec2,mag / arcsec2,mag,sigma,mag,erg / (A cm2 s),erg / (A cm2 s),erg / (A cm2 s),erg / (A cm2 s),erg / (A arcsec2 cm2 s),erg / (A arcsec2 cm2 s),erg / (A arcsec2 cm2 s),erg / (A arcsec2 cm2 s),erg / (A cm2 s),erg / (A cm2 s),mjy,mjy,mjy,mjy,mjy/arcsec2,mjy/arcsec2,mjy/arcsec2,mjy/arcsec2,mjy,mjy,mag,mag,mag,mag,mag / arcsec2,mag / arcsec2,mag / arcsec2,mag / arcsec2,mag,sigma,mag,erg / (A cm2 s),erg / (A cm2 s),erg / (A cm2 s),erg / (A cm2 s),erg / (A arcsec2 cm2 s),erg / (A arcsec2 cm2 s),erg / (A arcsec2 cm2 s),erg / (A arcsec2 cm2 s),erg / (A cm2 s),erg / (A cm2 s),mjy,mjy,mjy,mjy,mjy/arcsec2,mjy/arcsec2,mjy/arcsec2,mjy/arcsec2,mjy,mjy,ct / s,ct / s,,deg,deg,pix,pix,,s,,,s,s,,s,,s,,,,,,,,,,,,,,
float64,bytes12,float64,float64,float32,float32,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,bytes8,float64,float64,float32,float32,float32,float32,int16,bool,float32,float32,bytes11,float64,float64,float64,bool,bool,bool,bool,float64,float64,float64,float64,float64,float64,bytes2,bool,bool,bytes4
172587267.65609,U,172549649.16682,172624886.14536,655.1939,75236.98,0.0,78.54662,706.79254,78.54662,1.004,4437.703,64.11971,7365.4

In [7]:
###############
# Get subsets # 
###############

# Filter out 5 sigma outliers
xrt_rate_mean  = np.mean(tab[xrt_rate])
uvot_rate_mean = np.mean(tab[uvot_rate])

xrt_rate_std  = np.std(tab[xrt_rate])
uvot_rate_std = np.std(tab[uvot_rate])

tab_5_sig = tab[(tab[xrt_rate] < xrt_rate_mean + 5 * xrt_rate_std)
              & (tab[xrt_rate] > xrt_rate_mean - 5 * xrt_rate_std)
              & (tab[uvot_rate] < uvot_rate_mean + 5 * uvot_rate_std)
              & (tab[uvot_rate] > uvot_rate_mean - 5 * uvot_rate_std)]

# Get UL and BAD subsets
tab_UL     = tab[tab['UL'] == True]
tab_no_UL  = tab[tab['UL'] == False]

tab_BAD    = tab[tab['BAD'] == True]
tab_no_BAD = tab[tab['BAD'] == False]

tab_UL_no_bad = tab[(tab['BAD'] == False) & (tab['UL'] == True)]
tab_no_UL_no_bad = tab[(tab['BAD'] == False) & (tab['UL'] == False)]

In [8]:
tab

MET,EXTNAME,TSTART,TSTOP,EXPOSURE,TELAPSE,TIME,SRC_AREA,BKG_AREA,STD_AREA,PLATE_SCALE,RAW_TOT_CNTS,RAW_TOT_CNTS_ERR,RAW_BKG_CNTS,RAW_BKG_CNTS_ERR,RAW_STD_CNTS,RAW_STD_CNTS_ERR,RAW_TOT_RATE,RAW_TOT_RATE_ERR,RAW_BKG_RATE,RAW_BKG_RATE_ERR,RAW_STD_RATE,RAW_STD_RATE_ERR,COI_STD_FACTOR,COI_STD_FACTOR_ERR,COI_BKG_FACTOR,COI_BKG_FACTOR_ERR,COI_TOT_RATE,COI_TOT_RATE_ERR,COI_BKG_RATE,COI_BKG_RATE_ERR,COI_SRC_RATE,COI_SRC_RATE_ERR,AP_FACTOR,AP_FACTOR_ERR,AP_COI_SRC_RATE,AP_COI_SRC_RATE_ERR,LSS_FACTOR,LSS_RATE,LSS_RATE_ERR,SENSCORR_FACTOR,SENSCORR_RATE,SENSCORR_RATE_ERR,CORR_RATE,CORR_RATE_ERR,AB_MAG,AB_MAG_ERR,AB_MAG_ERR_STAT,AB_MAG_ERR_SYS,AB_MAG_BKG,AB_MAG_BKG_ERR_STAT,AB_MAG_BKG_ERR_SYS,AB_MAG_BKG_ERR,AB_MAG_LIM,AB_MAG_LIM_SIG,AB_MAG_COI_LIM,AB_FLUX_AA,AB_FLUX_AA_ERR_STAT,AB_FLUX_AA_ERR_SYS,AB_FLUX_AA_ERR,AB_FLUX_AA_BKG,AB_FLUX_AA_BKG_ERR_STAT,AB_FLUX_AA_BKG_ERR_SYS,AB_FLUX_AA_BKG_ERR,AB_FLUX_AA_LIM,AB_FLUX_AA_COI_LIM,AB_FLUX_HZ,AB_FLUX_HZ_ERR_STAT,AB_FLUX_HZ_ERR_SYS,AB_FLUX_HZ_ERR,AB_FLUX_HZ_BKG,AB_FLUX_HZ_BKG_ERR_STAT,AB_FLUX_HZ_BKG_ERR_SYS,AB_FLUX_HZ_BKG_ERR,AB_FLUX_HZ_LIM,AB_FLUX_HZ_COI_LIM,MAG,MAG_ERR,MAG_ERR_STAT,MAG_ERR_SYS,MAG_BKG,MAG_BKG_ERR_STAT,MAG_BKG_ERR_SYS,MAG_BKG_ERR,MAG_LIM,MAG_LIM_SIG,MAG_COI_LIM,FLUX_AA,FLUX_AA_ERR_STAT,FLUX_AA_ERR_SYS,FLUX_AA_ERR,FLUX_AA_BKG,FLUX_AA_BKG_ERR_STAT,FLUX_AA_BKG_ERR_SYS,FLUX_AA_BKG_ERR,FLUX_AA_LIM,FLUX_AA_COI_LIM,FLUX_HZ,FLUX_HZ_ERR_STAT,FLUX_HZ_ERR_SYS,FLUX_HZ_ERR,FLUX_HZ_BKG,FLUX_HZ_BKG_ERR_STAT,FLUX_HZ_BKG_ERR_SYS,FLUX_HZ_BKG_ERR,FLUX_HZ_LIM,FLUX_HZ_COI_LIM,COI_RATE_LIMIT,CORR_RATE_LIMIT,FILTER,RA,DEC,DETX,DETY,NSIGMA,FRAMTIME,SATURATED,SYS_ERR,SRCEXP,BKGEXP,OBSID,MJD_1,YEAR,MJD_0,FLAG_AB_MAG_99,FLAG_UPPER_LIM,FLAG_AB_MAG_ERR>2,FLAG_AB_MAG_>10_<30,MJD_2,T_+ve,T_-ve,Rate,Ratepos,Rateneg,MODE,BAD,UL,BAND
s,,s,s,s,s,s,arcsec2,arcsec2,arcsec2,arcsec / pix,ct,ct,ct,ct,ct,ct,ct / s,ct / s,ct / (arcsec2 s),ct / (arcsec2 s),ct / s,ct / s,,,,,ct / s,ct / s,ct / (arcsec2 s),ct / (arcsec2 s),ct / s,ct / s,,,ct / s,ct / s,,ct / s,ct / s,,ct / s,ct / s,ct / s,ct / s,mag,mag,mag,mag,mag / arcsec2,mag / arcsec2,mag / arcsec2,mag / arcsec2,mag,sigma,mag,erg / (A cm2 s),erg / (A cm2 s),erg / (A cm2 s),erg / (A cm2 s),erg / (A arcsec2 cm2 s),erg / (A arcsec2 cm2 s),erg / (A arcsec2 cm2 s),erg / (A arcsec2 cm2 s),erg / (A cm2 s),erg / (A cm2 s),mjy,mjy,mjy,mjy,mjy/arcsec2,mjy/arcsec2,mjy/arcsec2,mjy/arcsec2,mjy,mjy,mag,mag,mag,mag,mag / arcsec2,mag / arcsec2,mag / arcsec2,mag / arcsec2,mag,sigma,mag,erg / (A cm2 s),erg / (A cm2 s),erg / (A cm2 s),erg / (A cm2 s),erg / (A arcsec2 cm2 s),erg / (A arcsec2 cm2 s),erg / (A arcsec2 cm2 s),erg / (A arcsec2 cm2 s),erg / (A cm2 s),erg / (A cm2 s),mjy,mjy,mjy,mjy,mjy/arcsec2,mjy/arcsec2,mjy/arcsec2,mjy/arcsec2,mjy,mjy,ct / s,ct / s,,deg,deg,pix,pix,,s,,,s,s,,s,,s,,,,,,,,,,,,,,
float64,bytes12,float64,float64,float32,float32,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,bytes8,float64,float64,float32,float32,float32,float32,int16,bool,float32,float32,bytes11,float64,float64,float64,bool,bool,bool,bool,float64,float64,float64,float64,float64,float64,bytes2,bool,bool,bytes4
172587267.65609,U,172549649.16682,172624886.14536,655.1939,75236.98,0.0,78.54662,706.79254,78.54662,1.004,4437.703,64.11971,7365.4

In [9]:
###################################
# Perform lc sampling monte carlo # 
###################################

# Results dictionary
all_mc_res = []

for i in tqdm(range(N_mc)):
    mc_res = {}
    if include_bad == True:
        x_samp = np.random.normal(loc=tab_no_UL[xrt_rate], scale=tab_no_UL[xrt_rate_err])
        y_samp = np.random.normal(loc=tab_no_UL[uvot_rate], scale=tab_no_UL[uvot_rate_err])

    elif include_bad == False:
        x_samp = np.random.normal(loc=tab_no_UL_no_bad[xrt_rate], scale=tab_no_UL_no_bad[xrt_rate_err])
        y_samp = np.random.normal(loc=tab_no_UL_no_bad[uvot_rate], scale=tab_no_UL_no_bad[uvot_rate_err])

    # Add in the upper limit samples
    # We treat upper limit values as uniform between 0 and the value
    if include_UL:
        x_samp_UL = np.random.uniform(low=[0]*len(tab_UL), high=tab_UL[xrt_rate])
        y_samp_UL = np.random.uniform(low=[0]*len(tab_UL), high=tab_UL[uvot_rate])

        x_samp = np.concatenate([x_samp_UL, x_samp])
        y_samp = np.concatenate([y_samp_UL, y_samp])

    r, p_val = pearsonr(x_samp,y_samp)
    p_opt, p_cov = curve_fit(line, x_samp, y_samp)
    m, c = p_opt

    mc_res['r'] = r
    mc_res['m'] = m
    mc_res['c'] = c
    all_mc_res.append(mc_res)

df_mc_res = pd.DataFrame(all_mc_res)


100%|███████████████████████████████████████████████████████████████████████████| 10000/10000 [00:07<00:00, 1333.39it/s]


In [11]:
print('Linear Correlation Test...')
print('==========================')
print('--------------------------')

print('Input Parameters:')
print('-----------------')
print(f'include_bad : {include_bad}')
print(f'include_UL  : {include_UL}')
print(f'fits_path   : {fits_path}')

print('----------')
print('Settings:')
print('----------')
print(f'N_mc          = {N_mc}')
print(f'uvot_rate     = {uvot_rate}')
print(f'uvot_rate_err = {uvot_rate_err}')
print(f'xrt_rate      = {xrt_rate}')
print(f'xrt_rate_err  = {xrt_rate_err}')
print(f'uvot_filter   = {uvot_filter}')
print(f'xrt_curve     = {xrt_curve}')
print(f'simbad_name   = {simbad_name}')
print(f'readable_name = {readable_name}')

print('----------------')
print('Data Properties:')
print('----------------')
all_table_properties = [df_properties(tab, "tab"),
                        df_properties(tab_5_sig, "tab_5_sig"),
                        df_properties(tab_UL, "tab_UL"),
                        df_properties(tab_no_UL, "tab_no_UL"),
                        df_properties(tab_BAD, "tab_BAD"),
                        df_properties(tab_no_BAD, "tab_no_BAD"),
                        df_properties(tab_UL_no_bad, "tab_UL_no_bad"),
                        df_properties(tab_no_UL_no_bad, "tab_no_UL_no_bad")]
df_tab_prop = pd.DataFrame(all_table_properties)
print(df_tab_prop)


print('----------------------')
print('Excluded Observations:')
print('----------------------')
excluded_obs = {}
excluded_obs['tab_5_sig']        = get_exluded_obs(tab, tab_5_sig)
excluded_obs['tab_UL']           = get_exluded_obs(tab, tab_UL)
excluded_obs['tab_no_UL']        = get_exluded_obs(tab, tab_no_UL)
excluded_obs['tab_BAD']          = get_exluded_obs(tab, tab_BAD)
excluded_obs['tab_no_BAD']       = get_exluded_obs(tab, tab_no_BAD)
excluded_obs['tab_UL_no_bad']    = get_exluded_obs(tab, tab_UL_no_bad)
excluded_obs['tab_no_UL_no_bad'] = get_exluded_obs(tab, tab_no_UL_no_bad)
for k, v in excluded_obs.items():
    print(f'{k} : ')
    print(v)
    print('='*50)

print('-----------------------')
print('Monte Carlo Fit Results')
print('-----------------------')

print(df_mc_res)
#df_mc_res.to_csv(table_path/f'{outfile},line_mc.csv', index=False)

# Calculate mean fit values and errors
res['r_mean'] = df_mc_res['r'].mean()
res['r_std']  = df_mc_res['r'].std()
res['m_mean'] = df_mc_res['m'].mean()
res['m_std']  = df_mc_res['m'].std()
res['c_mean'] = df_mc_res['c'].mean()
res['c_std']  = df_mc_res['c'].std()

print('Best fit params:')
print(f'r = {res["r_mean"]:.2f} +- {res["r_std"]:.2f}')
print(f'm = {res["m_mean"]:.2f} +- {res["m_std"]:.2f}')
print(f'c = {res["c_mean"]:.2f} +- {res["c_std"]:.2f}')
      

Linear Correlation Test...
--------------------------
Input Parameters:
-----------------
include_bad : True
include_UL  : True
fits_path   : ../lightcurves/joined/NAME_NGC_1313_X-1,U,curve_nosys_join.fits
----------
Settings:
----------
N_mc          = 10000
uvot_rate     = COI_SRC_RATE
uvot_rate_err = COI_SRC_RATE_ERR
xrt_rate      = Rate
xrt_rate_err  = Ratepos
uvot_filter   = U
xrt_curve     = FULL
simbad_name   = NAME_NGC_1313_X-1
readable_name = NGC1313 X-1
----------------
Data Properties:
----------------
               name  length  N_obs  N_bad  N_good  N_UL
0               tab     288    281      2     286     2
1         tab_5_sig     285    278      2     283     2
2            tab_UL       2      2      1       1     2
3         tab_no_UL     286    279      1     285     0
4           tab_BAD       2      2      2       0     1
5        tab_no_BAD     286    279      0     286     1
6     tab_UL_no_bad       1      1      0       1     1
7  tab_no_UL_no_bad     285    27